<a href="https://colab.research.google.com/github/sangeetsaurabh/tweet_phrase_kaggle_competition/blob/master/pytorch_transformer/tweet_sentiment_xlm_roberta_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [1]:
!pip install transformers

     |████████████████████████████████| 757kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 13.2MB/s 
     |████████████████████████████████| 890kB 12.9MB/s 
     |████████████████████████████████| 3.0MB 24.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=c8e746442770131c81e267314b1d43d21f5ce655c44f8b00277e2dfabb8b2554
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import numpy as np
import pandas as pd
import os
import warnings
import random
import torch 
from torch import nn
import torch.optim as optim
from sklearn.model_selection import StratifiedKFold
import tokenizers
from transformers import RobertaModel, RobertaConfig, RobertaTokenizer
from transformers import XLMRobertaModel, XLMRobertaConfig, XLMRobertaTokenizer

warnings.filterwarnings('ignore')

# Seed

In [3]:
def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True

seed = 42
seed_everything(seed)

### Set up the path

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
data_folder = "/content/drive/My Drive/tweet-sentiment-extraction/data"
xlm_robert_folder = "/content/drive/My Drive/tweet-sentiment-extraction/xlm_robert/"
tmp_folder = '/tmp'

In [6]:
import shutil
shutil.copyfile('/content/drive/My Drive/tweet-sentiment-extraction/data/roberta-large-merges.txt', '/tmp/roberta-large-merges.txt')
shutil.copyfile('/content/drive/My Drive/tweet-sentiment-extraction/data/roberta-large-vocab.json', '/tmp/roberta-large-vocab.json')
shutil.copyfile('/content/drive/My Drive/tweet-sentiment-extraction/data/roberta-base-config.json', '/tmp/roberta-base-config.json')


'/tmp/roberta-base-config.json'

In [7]:
from transformers import RobertaTokenizer, RobertaForMaskedLM, XLMRobertaTokenizer
import torch

tokenizer3 = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base', lowercase=True, add_prefix_space=True)
print(tokenizer3.decode(tokenizer3.encode("I am the king", "Positive")))
print(tokenizer3.encode("I am the king", "Positive"))





<s> I am the king</s></s> Positive</s>
[0, 87, 444, 70, 60097, 2, 2, 156060, 13, 2]


In [8]:
tokenizer3.vocab_size

250002

# Data Loader

In [9]:
!pip install protobuf
!wget https://raw.githubusercontent.com/google/sentencepiece/master/python/sentencepiece_pb2.py
!pip install sentencepiece

--2020-07-03 17:58:24--  https://raw.githubusercontent.com/google/sentencepiece/master/python/sentencepiece_pb2.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7382 (7.2K) [text/plain]
Saving to: ‘sentencepiece_pb2.py’

sentencepiece_pb2.p 100%[===================>]   7.21K  --.-KB/s    in 0s      

2020-07-03 17:58:25 (76.6 MB/s) - ‘sentencepiece_pb2.py’ saved [7382/7382]



In [10]:
!wget https://s3.amazonaws.com/models.huggingface.co/bert/xlm-roberta-base-sentencepiece.bpe.model

--2020-07-03 17:58:30--  https://s3.amazonaws.com/models.huggingface.co/bert/xlm-roberta-base-sentencepiece.bpe.model
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.49.75
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.49.75|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5069051 (4.8M) [binary/octet-stream]
Saving to: ‘xlm-roberta-base-sentencepiece.bpe.model’

xlm-roberta-base-se 100%[===================>]   4.83M  3.29MB/s    in 1.5s    

2020-07-03 17:58:32 (3.29 MB/s) - ‘xlm-roberta-base-sentencepiece.bpe.model’ saved [5069051/5069051]



In [11]:
import sentencepiece_pb2
import sentencepiece as spm
class SentencePieceTokenizer:
    def __init__(self):
        self.sp = spm.SentencePieceProcessor()
        #self.sp.load(os.path.join(model_path, "spiece.model"))
        self.sp.load("xlm-roberta-base-sentencepiece.bpe.model")
    
    def encode(self, sentence):
        spt = sentencepiece_pb2.SentencePieceText()
        spt.ParseFromString(self.sp.encode_as_serialized_proto(sentence))
        offsets = []
        tokens = []
        for piece in spt.pieces:
            tokens.append(piece.id)
            offsets.append((piece.begin, piece.end))
        return tokens, offsets

In [12]:
tokenizer1 = SentencePieceTokenizer()

In [13]:
tokenizer1.encode("positive")
[0, 87, 444, 70, 60097, 2, 2, 156060, 13, 2]

[0, 87, 444, 70, 60097, 2, 2, 156060, 13, 2]

In [14]:
tokenizer1.sp.id_to_piece(87)

'▁el'

In [15]:
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, df, max_len=96):
        self.df = df
        self.max_len = max_len
        self.labeled = 'selected_text' in df
        self.tokenizer1 = SentencePieceTokenizer()
        #self.tokenizer2 = tokenizers.ByteLevelBPETokenizer(
        #    vocab_file='/tmp/roberta-large-vocab.json', 
        #    merges_file='/tmp/roberta-large-merges.txt', 
        #    lowercase=True,
        #    add_prefix_space=True)

    def __getitem__(self, index):
        data = {}
        row = self.df.iloc[index]
        
        ids, masks, tweet, offsets,row_id = self.get_input_data(row)

        data['ids'] = ids
        data['masks'] = masks
        data['tweet'] = tweet
        data['offsets'] = offsets
        data['row_id'] = row_id
        
        if self.labeled:
            #print("I came here")
            start_idx, end_idx = self.get_target_idx(row, tweet, offsets)
            data['start_idx'] = start_idx
            data['end_idx'] = end_idx
        
        return data

    def __len__(self):
        return len(self.df)
    
    def get_input_data(self, row):
        tweet = " " + " ".join(row.text.lower().split())
        encoding_ids, encoding_offsets = self.tokenizer1.encode(tweet)
        sentiment_id, _ = self.tokenizer1.encode(row.sentiment)
        ids = [0] + sentiment_id + [2, 2] + encoding_ids + [2]
        offsets = [(0, 0)] * 4 + encoding_offsets + [(0, 0)]
                
        pad_len = self.max_len - len(ids)
        if pad_len > 0:
            ids += [1] * pad_len
            offsets += [(0, 0)] * pad_len
        
        ids = torch.tensor(ids)
        masks = torch.where(ids != 1, torch.tensor(1), torch.tensor(0))
        offsets = torch.tensor(offsets)
        
        return ids, masks, tweet, offsets, row.idx
        
    def get_target_idx(self, row, tweet, offsets):
        selected_text = " " +  " ".join(row.selected_text.lower().split())
        #print(selected_text)
        #print(row.idx)

        len_st = len(selected_text) - 1
        idx0 = None
        idx1 = None

        for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
            if " " + tweet[ind: ind+len_st] == selected_text:
                idx0 = ind
                idx1 = ind + len_st - 1
                break

        char_targets = [0] * len(tweet)
        if idx0 != None and idx1 != None:
            for ct in range(idx0, idx1 + 1):
                char_targets[ct] = 1

        target_idx = []
        for j, (offset1, offset2) in enumerate(offsets):
            if sum(char_targets[offset1: offset2]) > 0:
                target_idx.append(j)

        start_idx = target_idx[0]
        end_idx = target_idx[-1]
        
        return start_idx, end_idx
        
def get_train_val_loaders(df, train_idx, val_idx, batch_size=8):
    train_df = df.iloc[train_idx]
    val_df = df.iloc[val_idx]

    train_loader = torch.utils.data.DataLoader(
        TweetDataset(train_df), 
        batch_size=batch_size, 
        shuffle=True, 
        num_workers=2,
        drop_last=True)

    val_loader = torch.utils.data.DataLoader(
        TweetDataset(val_df), 
        batch_size=batch_size, 
        shuffle=False, 
        num_workers=2)

    dataloaders_dict = {"train": train_loader, "val": val_loader}

    return dataloaders_dict

def get_test_loader(df, batch_size=32):
    loader = torch.utils.data.DataLoader(
        TweetDataset(df), 
        batch_size=batch_size, 
        shuffle=False, 
        num_workers=2)    
    return loader

# Model

In [16]:
class TweetModel(nn.Module):
    def __init__(self):
        super(TweetModel, self).__init__()
        config = XLMRobertaConfig.from_pretrained(
            'xlm-roberta-base', output_hidden_states=True)    
        self.roberta = XLMRobertaModel.from_pretrained(
            'xlm-roberta-base', config=config)
        #config = RobertaConfig.from_pretrained(
        #    'roberta-large', output_hidden_states=True)    
        #self.roberta = RobertaModel.from_pretrained(
        #    'roberta-large', config=config)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(config.hidden_size, 2)
        nn.init.normal_(self.fc.weight, std=0.02)
        nn.init.normal_(self.fc.bias, 0)

    def forward(self, input_ids, attention_mask):
        _, _, hs = self.roberta(input_ids, attention_mask)
         
        x = torch.stack([hs[-1], hs[-2], hs[-3], hs[-4]])
        x = torch.mean(x, 0)
        x = self.dropout(x)
        x = self.fc(x)
        start_logits, end_logits = x.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)
                
        return start_logits, end_logits

# Loss Function

In [17]:
def loss_fn(start_logits, end_logits, start_positions, end_positions):
    ce_loss = nn.CrossEntropyLoss()
    start_loss = ce_loss(start_logits, start_positions)
    end_loss = ce_loss(end_logits, end_positions)    
    total_loss = start_loss + end_loss
    return total_loss

# Evaluation Function

In [18]:
def get_selected_text(text, start_idx, end_idx, offsets):
    selected_text = ""
    for ix in range(start_idx, end_idx + 1):
        selected_text += text[offsets[ix][0]: offsets[ix][1]]
        if (ix + 1) < len(offsets) and offsets[ix][1] < offsets[ix + 1][0]:
            selected_text += " "
    return selected_text

def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

def compute_jaccard_score(text, start_idx, end_idx, start_logits, end_logits, offsets):
    start_pred = np.argmax(start_logits)
    end_pred = np.argmax(end_logits)
    if start_pred > end_pred:
        pred = text
    else:
        pred = get_selected_text(text, start_pred, end_pred, offsets)
        
    true = get_selected_text(text, start_idx, end_idx, offsets)
    
    return jaccard(true, pred)

# Training Function

In [19]:
def train_model(model, dataloaders_dict, criterion, optimizer, num_epochs, filename):
    model.cuda()

    for epoch in range(num_epochs):
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            epoch_loss = 0.0
            epoch_jaccard = 0.0
            
            for data in (dataloaders_dict[phase]):
                ids = data['ids'].cuda()
                masks = data['masks'].cuda()
                tweet = data['tweet']
                offsets = data['offsets'].numpy()
                start_idx = data['start_idx'].cuda()
                end_idx = data['end_idx'].cuda()

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):

                    start_logits, end_logits = model(ids, masks)

                    loss = criterion(start_logits, end_logits, start_idx, end_idx)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    epoch_loss += loss.item() * len(ids)
                    
                    start_idx = start_idx.cpu().detach().numpy()
                    end_idx = end_idx.cpu().detach().numpy()
                    start_logits = torch.softmax(start_logits, dim=1).cpu().detach().numpy()
                    end_logits = torch.softmax(end_logits, dim=1).cpu().detach().numpy()
                    
                    for i in range(len(ids)):                        
                        jaccard_score = compute_jaccard_score(
                            tweet[i],
                            start_idx[i],
                            end_idx[i],
                            start_logits[i], 
                            end_logits[i], 
                            offsets[i])
                        epoch_jaccard += jaccard_score
                    
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_jaccard = epoch_jaccard / len(dataloaders_dict[phase].dataset)
            
            print('Epoch {}/{} | {:^5} | Loss: {:.4f} | Jaccard: {:.4f}'.format(
                epoch + 1, num_epochs, phase, epoch_loss, epoch_jaccard))
    
    torch.save(model.state_dict(), filename)

# Training

In [20]:
num_epochs = 3
batch_size = 32
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

In [21]:
train_df = pd.read_csv(data_folder + '/train.csv')
train_df['text'] = train_df['text'].astype(str)
train_df['selected_text'] = train_df['selected_text'].astype(str)
train_df['idx'] = train_df.index.copy()

for fold, (train_idx, val_idx) in enumerate(skf.split(train_df, train_df.sentiment), start=1): 
    print(f'Fold: {fold}')

    model = TweetModel()
    optimizer = optim.AdamW(model.parameters(), lr=3e-5, betas=(0.9, 0.999))
    criterion = loss_fn    
    dataloaders_dict = get_train_val_loaders(train_df, train_idx, val_idx, batch_size)
    #print(dataloaders_dict)


    train_model(
        model, 
        dataloaders_dict,
        criterion, 
        optimizer, 
        num_epochs,
        xlm_robert_folder + f'roberta_fold{fold}.pth')

Fold: 1



Epoch 1/3 | train | Loss: 2.9527 | Jaccard: 0.6035
Epoch 1/3 |  val  | Loss: 2.2656 | Jaccard: 0.6453
Epoch 2/3 | train | Loss: 2.2483 | Jaccard: 0.6528
Epoch 2/3 |  val  | Loss: 2.1182 | Jaccard: 0.6593
Epoch 3/3 | train | Loss: 2.0476 | Jaccard: 0.6710
Epoch 3/3 |  val  | Loss: 2.0333 | Jaccard: 0.6638
Fold: 2
Epoch 1/3 | train | Loss: 2.9759 | Jaccard: 0.6062
Epoch 1/3 |  val  | Loss: 2.3006 | Jaccard: 0.6479
Epoch 2/3 | train | Loss: 2.2204 | Jaccard: 0.6561
Epoch 2/3 |  val  | Loss: 2.1119 | Jaccard: 0.6678
Epoch 3/3 | train | Loss: 2.0195 | Jaccard: 0.6757
Epoch 3/3 |  val  | Loss: 2.0506 | Jaccard: 0.6751
Fold: 3
Epoch 1/3 | train | Loss: 2.9129 | Jaccard: 0.6074
Epoch 1/3 |  val  | Loss: 2.2820 | Jaccard: 0.6509
Epoch 2/3 | train | Loss: 2.1698 | Jaccard: 0.6607
Epoch 2/3 |  val  | Loss: 2.0825 | Jaccard: 0.6677
Epoch 3/3 | train | Loss: 1.9534 | Jaccard: 0.6813
Epoch 3/3 |  val  | Loss: 2.0722 | Jaccard: 0.6770
Fold: 4
Epoch 1/3 | train | Loss: 2.9105 | Jaccard: 0.6079
Epoch 

In [22]:
train_df.head()

,textID,text,selected_text,sentiment,idx
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,0
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,1
2,088c60f138,my boss is bullying me...,bullying me,negative,2
3,9642c003ef,what interview! leave me alone,leave me alone,negative,3
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,4


# Inference

In [21]:
%%time

test_df = pd.read_csv(data_folder + '/test.csv')
test_df['text'] = test_df['text'].astype(str)
test_df['idx'] = test_df.index.copy()
test_loader = get_test_loader(test_df)
predictions = []
models = []
for fold in range(skf.n_splits):
    model = TweetModel()
    model.cuda()
    model.load_state_dict(torch.load(xlm_robert_folder + f'roberta_fold{fold+1}.pth'))
    model.eval()
    models.append(model)

for data in test_loader:
    ids = data['ids'].cuda()
    masks = data['masks'].cuda()
    tweet = data['tweet']
    offsets = data['offsets'].numpy()

    start_logits = []
    end_logits = []
    for model in models:
        with torch.no_grad():
            output = model(ids, masks)
            start_logits.append(torch.softmax(output[0], dim=1).cpu().detach().numpy())
            end_logits.append(torch.softmax(output[1], dim=1).cpu().detach().numpy())

    start_logits = np.mean(start_logits, axis=0)
    end_logits = np.mean(end_logits, axis=0)
    for i in range(len(ids)):    
        start_pred = np.argmax(start_logits[i])
        end_pred = np.argmax(end_logits[i])
        if start_pred > end_pred:
            pred = tweet[i]
        else:
            pred = get_selected_text(tweet[i], start_pred, end_pred, offsets[i])
        predictions.append(pred)


CPU times: user 3min 55s, sys: 55.6 s, total: 4min 50s
Wall time: 6min 54s


# Submission

In [23]:
sub_df = pd.read_csv(data_folder + '/sample_submission.csv')
sub_df['selected_text'] = predictions
sub_df['selected_text'] = sub_df['selected_text'].apply(lambda x: x.replace('!!!!', '!') if len(x.split())==1 else x)
sub_df['selected_text'] = sub_df['selected_text'].apply(lambda x: x.replace('..', '.') if len(x.split())==1 else x)
sub_df['selected_text'] = sub_df['selected_text'].apply(lambda x: x.replace('...', '.') if len(x.split())==1 else x)
sub_df.to_csv('submission.csv', index=False)
sub_df.head()

,textID,selected_text
0,f87dea47db,last session of the day
1,96d74cb729,shanghai is also really exciting (precisely -...
2,eee518ae67,shame!
3,01082688c6,happy
4,33987a8ee5,i like it!!
